In [1]:
import pandas as pd 
import numpy as np 
from datetime import date
from optimisation_utiles import * 
df = pd.read_csv('ARP_Returns.csv')[:3026]
df_return = cal_return(df)
# My weekly change
df_weekly = df_return.resample('W-FRI').apply(lambda x: (1+x).prod()-1)

In [2]:
df_weekly = df_weekly[df_weekly.index>'2009-09-01']
df_weekly = df_weekly.drop(columns=['equity_momentum', 'equity_low_beta', 'equity_quality', 'equity_trend', 'fx_value', 'commodity_carry'])
df_return = df_return[df_return.index>'2009-09-01']
df_return = df_return.drop(columns=['equity_momentum', 'equity_low_beta', 'equity_quality', 'equity_trend', 'fx_value', 'commodity_carry'])


In [2]:
%%time
div_results = rolling_portfolio_optimisation(df_return=df_weekly, default_upper_bound=0.1, default_lower_bound=0, window_size=36, optimisation_freq=12, target='diversification_ratio', input_bounds=None, product='ARP')

2009-03-29 ---- 2012-03-30
2010-03-28 ---- 2013-03-29
2011-03-27 ---- 2014-03-30
2012-03-30 ---- 2015-03-29
2013-03-29 ---- 2016-03-27
2014-03-30 ---- 2017-03-31
2015-03-29 ---- 2018-03-30
2016-03-27 ---- 2019-03-29
2017-03-31 ---- 2020-03-29
CPU times: user 1min 8s, sys: 885 ms, total: 1min 9s
Wall time: 18 s


In [3]:
input_bounds_max_sharpe = Duncans_weights(df_weekly, div_results)

2012-03-30
2013-03-29
2014-03-30
2015-03-29
2016-03-27
2017-03-31
2018-03-30
2019-03-29
2020-03-29


In [4]:
# cap vol carry at 0.3
for date in input_bounds_max_sharpe:
    input_bounds_max_sharpe[date]['interest_rate_volatility_carry'] = [0, 0.03]
    input_bounds_max_sharpe[date]['commodity_volatility_carry'] = [0, 0.03]
    input_bounds_max_sharpe[date]['equity_volatility_carry'] = [0, 0.03]

In [5]:
sharpe_results = rolling_portfolio_optimisation(df_return=df_weekly, default_upper_bound=1, default_lower_bound=0, window_size=36, optimisation_freq=12, target='sharpe_ratio', input_bounds=input_bounds_max_sharpe, product='ARP', optimisation_method='BFGS')
df_component_weights_s, df_weights_s = generate_component_weights(df_weekly, sharpe_results)
df_component_weights_d, df_weights_d = generate_component_weights(df_weekly, div_results)

2009-03-29 ---- 2012-03-30
2010-03-28 ---- 2013-03-29
2011-03-27 ---- 2014-03-30
2012-03-30 ---- 2015-03-29
2013-03-29 ---- 2016-03-27
2014-03-30 ---- 2017-03-31
2015-03-29 ---- 2018-03-30
2016-03-27 ---- 2019-03-29
2017-03-31 ---- 2020-03-29


In [6]:
from PortfolioPerformance import * 
rebalance_results_s = Get_Portfolio_Index_Base_Case(df_return, df_component_weights_s, initial_notional=100)
rebalance_results_d = Get_Portfolio_Index_Base_Case(df_return, df_component_weights_d, initial_notional=100)

In [8]:
df_component_return_s = rebalance_results_s['PnL_df_net']
df_component_return_s = df_component_return_s[(df_component_return_s.index >= '2012-09-29')]
df_component_return_d = rebalance_results_d['PnL_df_net']
df_component_return_d = df_component_return_d[(df_component_return_d.index >= '2012-09-29')]

In [7]:
full_stats_d = save_results(div_results, rebalance_results_d, which='div', df_weekly=df_weekly)
full_stats_d

NameError: name 'rolling_results' is not defined

In [ ]:
full_stats_s = save_results(sharpe_results, rebalance_results_s, which='sharpe')
full_stats_s